In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from os import path

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_dir = '../input/fruit-and-vegetable-image-recognition/train'
valid_dir = '../input/fruit-and-vegetable-image-recognition/validation'
test_dir = '../input/fruit-and-vegetable-image-recognition/test'


In [ ]:
totalFiles = 0

for root, dirs, files in os.walk(train_dir):
        for Files in files:
            totalFiles += 1

        
print('total files:', totalFiles)

In [ ]:
def count_files(path):
    for root, dirs, files in os.walk(path):
        print("{} files in {}".format(len(files), root))

count_files(train_dir)

In [ ]:
print("On average there are", round(sum([len(files) for r, d, files in os.walk(train_dir)]) / sum([len(d) for r, d, files in os.walk(train_dir)]), 2), "images for each class in the training set.")


In [ ]:
def get_subdirname(path):
    return [name for name in os.listdir(path) # listdir() returns a list containing the names of the entries in the directory given by path. The list is in arbitrary order.
            if os.path.isdir(os.path.join(path, name))]

class_names = get_subdirname(train_dir)
class_names

In [ ]:
def transform_ds(path):
    # label_mode='categorical' means that the labels are encoded as a categorical vector (e.g. for categorical_crossentropy loss).
    tf_ds = tf.keras.utils.image_dataset_from_directory(path, labels='inferred', label_mode='categorical', 
                                                        image_size=(128, 128), batch_size=32, seed=42)
    
    print("tf.data.dataset from image files in directory created.")
    
    return tf_ds

In [ ]:
train_ds = transform_ds(train_dir)
valid_ds = transform_ds(valid_dir)
test_ds = transform_ds(test_dir)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(27, 20))

for image, label in train_ds.take(1):
    for i in range(10):
        ax = plt.subplot(1, 10, i+1)
        plt.imshow(image[i].numpy().astype('int'))
        plt.title(train_ds.class_names[tf.argmax(label[i])])

In [ ]:
# augment images a little bit
data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal_and_vertical'),
    layers.RandomRotation(0.2),
])

# rescale images
rescaling = keras.Sequential([
    layers.Rescaling(1./255) # standardizes pixel values to the [0, 1] range.
])

In [ ]:
plt.figure(figsize=(27, 20))
for image, label in train_ds.take(1):
    for i in range(10):
        augmented_image = data_augmentation(image)
        ax = plt.subplot(1, 10, i+1)
        plt.imshow(augmented_image[0].numpy().astype('uint8'))
        plt.axis('off')

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape) # batch = 32
    print(labels_batch.shape)
    
    break

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
res50 = ResNet50(include_top=False, weights = 'imagenet', input_shape=(128, 128, 3))
res50.trainable = False

In [ ]:
model = Sequential()
model.add(res50)
model.add(layers.Flatten())
model.add(layers.Activation('relu'))
model.add(layers.Dense(36))
model.add(layers.Activation('softmax'))


model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, batch_size=32, epochs=15, validation_data=valid_ds, callbacks=[EarlyStopping(monitor='val_accuracy', patience=3)])

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.grid(True)
plt.legend(['accuracy', 'val_accuracy'])
plt.xlabel('Epochs')
plt.show()

plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(['loss', 'val_loss'])
plt.grid()
plt.xlabel('Epochs')

In [ ]:
model.metrics_names 

In [ ]:
model.evaluate(train_ds, verbose=1)

In [ ]:
test_labels = test_ds.class_names

plt.figure(figsize=(30,25))

for image, label in train_ds.take(1):
    model_prediction = model.predict(image)
    for i in range(10):
        plt.subplot(1, 10, i+1)
        plt.imshow(image[i].numpy().astype('int'))
        plt.title(f"Prediction: {test_labels[tf.argmax(tf.round(model_prediction[i]))]}\n Original Labels: {test_labels[tf.argmax(label[i])]}")
        plt.grid(True)
        plt.axis('off')
        
    plt.show()
